In [1]:
import os 
from pathlib import Path

In [2]:
%pwd

'/home/priyanshu1303d/Projects/DeepQA_PyTorch/research'

In [3]:
os.chdir("../")

In [5]:
%pwd

'/home/priyanshu1303d/Projects/DeepQA_PyTorch'

In [6]:
from dataclasses import dataclass

In [19]:
@dataclass(frozen=True)
class ModelPredictionConfig:
    saved_model_path: Path
    vocab_file_path: Path


In [11]:
from DeepQA.constants import *
from DeepQA.utils.common import *

In [20]:
class ConfigurationManager:
    def __init__(self , config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_model_prediction_config(self) -> ModelPredictionConfig:
        config = self.config.model_prediction

        model_prediction_config = ModelPredictionConfig(
            saved_model_path= config.saved_model_path,
            vocab_file_path = config.vocab_file_path
        )
        return model_prediction_config

In [21]:
import torch
import json
import argparse
from pathlib import Path
from DeepQA.utils.common import read_yaml
import torch.nn as nn

In [22]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=50, hidden_size=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x):
        x = self.embedding(x) 

        # 🚨 Fix: Remove the extra dimension if needed
        if x.dim() == 4:  
            x = x.squeeze(1)  # Remove the unnecessary 1-dim (batch_size, 1, seq_len, embedding_dim) → (batch_size, seq_len, embedding_dim)
        
        output, hidden = self.rnn(x)  # Pass through RNN
        output = self.fc(output[:, -1, :])  # Take the last output for classification

        return output

class Predictor:
    def __init__(self, config: ModelPredictionConfig):
        """Initialize predictor with model and vocab."""
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load vocabulary
        with open(self.config.vocab_file_path, "r") as f:
            self.vocab = json.load(f)
        
        self.vocab_size = len(self.vocab)

        # Load model
        self.model = RNNModel(vocab_size=self.vocab_size).to(self.device)
        self.model.load_state_dict(torch.load(self.config.saved_model_path, map_location=self.device))
        self.model.eval()

        # Reverse vocab for decoding predictions
        self.index_to_word = {idx: word for word, idx in self.vocab.items()}

    def preprocess_text(self, text: str):
        """Convert input text into numericalized tensor."""
        numerical_input = [self.vocab.get(word, self.vocab.get("<UNK>", 0)) for word in text.split()]
        return torch.tensor(numerical_input, dtype=torch.long).unsqueeze(0).to(self.device)

    def predict(self, text: str):
        """Generate prediction from input text."""
        input_tensor = self.preprocess_text(text)

        with torch.no_grad():
            output = self.model(input_tensor)
            predicted_index = torch.argmax(output, dim=1).item()

        predicted_word = self.index_to_word.get(predicted_index, "<UNK>")
        return predicted_word

In [37]:
try:
    config = ConfigurationManager()
    get_model_prediction_config  = config.get_model_prediction_config()
    model_prediction = Predictor(get_model_prediction_config)
    answer = model_prediction.predict("What is the largest mammal on Earth?")
    print(answer)
except Exception as e:
    raise e

[2025-04-03 21:24:45,052 : INFO : common  : yaml file config/config.yaml was read succesfully]
[2025-04-03 21:24:45,054 : INFO : common  : yaml file params.yaml was read succesfully]
whale
